https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2025/homework%202.md

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Question 1: [IPO] Withdrawn IPOs by Company Type

What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

In [13]:
# load the IPO withdrawal table into a DataFrame
url = 'https://stockanalysis.com/ipos/withdrawn/'
data = pd.read_html(url)
df = data[0]
print(df.shape)
df.head()

(100, 4)


,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


Create a new column called Company Class, categorizing company names based on patterns like:
* “Acquisition Corp” or “Acquisition Corporation” → Acq.Corp
* “Inc” or “Incorporated” → Inc
* “Group” → Group
* “Ltd” or “Limited” → Limited
* “Holdings” → Holdings
* Others → Other

In [ ]:
def get_company_class(s):
    if s.find('Acquisition Corp') > 0 or s.find('Acquisition Corporation') > 0:
        return 'Acq.Corp'
    elif s.find('Inc') > 0 or s.find('Incorporated') > 0:
        return 'Inc'
    elif s.find('Group') > 0:
        return 'Group'
    elif s.find('Ltd') > 0 or s.find('Limited') > 0:
        return 'Limited'
    elif s.find('Holdings') > 0:
        return 'Holdings'
    else:
        return 'Other'

In [35]:
df['Company Class'] = df['Company Name'].apply(get_company_class)

In [37]:
df.head(20)

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
5,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000,Limited
6,CABR,"Caring Brands, Inc.",$4.00,750000,Inc
7,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000,Inc
8,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000,Limited
9,KMCM,Key Mining Corp.,$2.25,4444444,Other
